## Install TensorTrade

In [38]:
#!python -m pip install -e .. -U

## Setup Data Fetching

In [39]:
import ssl
import pandas as pd

from tensortrade.utils import CryptoDataDownload

ssl._create_default_https_context = ssl._create_unverified_context # Only used if pandas gives a SSLError

cdd = CryptoDataDownload()

In [40]:
data = pd.concat([
    cdd.fetch("Coinbase", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Coinbase", "USD", "ETH", "1h").add_prefix("ETH:")
], axis=1)
data = data.drop(["ETH:date"], axis=1)
data = data.rename({"BTC:date": "date"}, axis=1)

In [41]:
data.head()

,date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,ETH:open,ETH:high,ETH:low,ETH:close,ETH:volume
0,2017-07-01 11:00:00,2505.56,2513.38,2495.12,2509.17,287000.32,279.98,279.99,272.1,275.01,679358.87
1,2017-07-01 12:00:00,2509.17,2512.87,2484.99,2488.43,393142.50,275.01,275.01,271.0,274.83,824362.87
2,2017-07-01 13:00:00,2488.43,2488.43,2454.40,2454.43,693254.01,274.83,274.93,265.0,268.79,3010787.99
3,2017-07-01 14:00:00,2454.43,2473.93,2450.83,2459.35,712864.80,268.79,269.90,265.0,265.74,1702536.85
4,2017-07-01 15:00:00,2459.35,2475.00,2450.00,2467.83,682105.41,265.74,272.74,265.0,272.57,1500282.55


## Create features with the data module

In [42]:
from tensortrade.data import Node, Module, DataFeed, Stream, Select


def rsi(price: Node, period: float):
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Node, fast: float, slow: float, signal: float) -> Node:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream(list(data[c])).rename(data[c].name)
    features += [s]

btc_close = Select("BTC:close")(*features)
eth_close = Select("ETH:close")(*features)

features += [
    rsi(btc_close, period=20).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
    rsi(eth_close, period=20).rename("ETH:rsi"),
    macd(eth_close, fast=10, slow=50, signal=5).rename("ETH:macd")
]
        
feed = DataFeed(features)
feed.compile()

In [43]:
feed.next()

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:933: RuntimeWarning: invalid value encountered in double_scalars
  v = (w[::-1] * x).sum() / w.sum()


{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'ETH:open': 279.98,
 'ETH:high': 279.99,
 'ETH:low': 272.1,
 'ETH:close': 275.01,
 'ETH:volume': 679358.87,
 'BTC:rsi': nan,
 'BTC:macd': 0.0,
 'ETH:rsi': nan,
 'ETH:macd': 0.0}

## Setup Trading Environment

In [65]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns

coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(data["BTC:close"])).rename("USD-BTC"),
    Stream(list(data["ETH:close"])).rename("USD-ETH")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 0 * BTC),
    Wallet(coinbase, 0 * ETH),
])

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino'),
    window_size=20
)

## Example Data Feed Observation

Even though this observation contains data from the internal data feed, since `use_internal=False` this data will not be provided as input to the observation history. The data that will be added to observation history of the environment will strictly be the nodes that have been included into the data feed that has been provided as a parameter to the trading environment.

In [45]:
env.feed.next()

{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'ETH:open': 279.98,
 'ETH:high': 279.99,
 'ETH:low': 272.1,
 'ETH:close': 275.01,
 'ETH:volume': 679358.87,
 'BTC:rsi': 0.0,
 'BTC:macd': -0.23222985476439617,
 'ETH:rsi': 0.0,
 'ETH:macd': -0.0020154953644232945,
 'coinbase:/USD-BTC': 2509.17,
 'coinbase:/USD-ETH': 275.01,
 'coinbase:/USD:/free': 10000.0,
 'coinbase:/USD:/locked': 0.0,
 'coinbase:/USD:/total': 10000.0,
 'coinbase:/BTC:/free': 0.0,
 'coinbase:/BTC:/locked': 0.0,
 'coinbase:/BTC:/total': 0.0,
 'coinbase:/BTC:/worth': 0.0,
 'coinbase:/ETH:/free': 0.0,
 'coinbase:/ETH:/locked': 0.0,
 'coinbase:/ETH:/total': 0.0,
 'coinbase:/ETH:/worth': 0.0,
 'net_worth': 10000.0}

## Setup and Train LSTM Agent

In [ ]:
import random
import numpy as np
import tensorflow as tf

from collections import namedtuple

from tensortrade.agents import Agent, ReplayMemory

A2CTransition = namedtuple('A2CTransition', ['state', 'action', 'reward', 'done', 'value'])


class A2C_LSTMAgent_DEV(Agent):

    def __init__(self,
                 env: 'TradingEnvironment',
                 shared_network: tf.keras.Model = None,
                 actor_network: tf.keras.Model = None,
                 critic_network: tf.keras.Model = None):
        self.env = env
        self.n_actions = env.action_space.n
        self.observation_shape = env.observation_space.shape

        self.shared_network = shared_network or self._build_shared_network()
        self.actor_network = actor_network or self._build_actor_network()
        self.critic_network = critic_network or self._build_critic_network()

        self.env.agent_id = self.id

    def _build_shared_network(self):
        network = tf.keras.Sequential([
            tf.keras.layers.TimeDistributed(tf.keras.layers.InputLayer(input_shape=self.observation_shape)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=64, kernel_size=6, padding="same", activation="tanh")),
            tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(pool_size=2)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding="same", activation="tanh")),
            tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling1D(pool_size=2)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
            tf.keras.layers.LSTM(50, return_sequences=True, stateful=True)

        ])
        return network

    def _build_actor_network(self):
        actor_head = tf.keras.Sequential([
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dense(self.n_actions, activation='relu')
        ])
        #return tf.keras.Sequential([self.shared_network])
        return tf.keras.Sequential([self.shared_network, actor_head])

    def _build_critic_network(self):
        critic_head = tf.keras.Sequential([
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.Dense(25, activation='relu'),
            tf.keras.layers.Dense(1, activation='relu')
        ])
        #return tf.keras.Sequential([self.shared_network])

        return tf.keras.Sequential([self.shared_network, critic_head])

    def restore(self, path: str, **kwargs):
        actor_filename: str = kwargs.get('actor_filename', None)
        critic_filename: str = kwargs.get('critic_filename', None)

        if not actor_filename or not critic_filename:
            raise ValueError(
                'The `restore` method requires a directory `path`, a `critic_filename`, and an `actor_filename`.')

        self.actor_network = tf.keras.models.load_model(path + actor_filename)
        self.critic_network = tf.keras.models.load_model(path + critic_filename)

    def save(self, path: str, **kwargs):
        episode: int = kwargs.get('episode', None)

        if episode:
            suffix = self.id + "__" + str(episode).zfill(3) + ".hdf5"
            actor_filename = "actor_network__" + suffix
            critic_filename = "critic_network__" + suffix
        else:
            actor_filename = "actor_network__" + self.id + ".hdf5"
            critic_filename = "critic_network__" + self.id + ".hdf5"

        self.actor_network.save(path + actor_filename)
        self.critic_network.save(path + critic_filename)

    def get_action(self, state: np.ndarray, **kwargs) -> int:
        threshold: float = kwargs.get('threshold', 0)

        rand = random.random()

        if rand < threshold:
            return np.random.choice(self.n_actions)
        else:
            logits = self.actor_network(state[None, None, :], training=False)
            return tf.squeeze(tf.squeeze(tf.random.categorical(logits[0], 1), axis=-1), axis=-1)

    def _apply_gradient_descent(self,
                                memory: ReplayMemory,
                                batch_size: int,
                                learning_rate: float,
                                discount_factor: float,
                                entropy_c: float,):
        huber_loss = tf.keras.losses.Huber()
        wsce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

        transitions = memory.tail(batch_size)
        batch = A2CTransition(*zip(*transitions))

        states = tf.convert_to_tensor(batch.state)
        actions = tf.convert_to_tensor(batch.action)
        rewards = tf.convert_to_tensor(batch.reward, dtype=tf.float32)
        dones = tf.convert_to_tensor(batch.done)
        values = tf.convert_to_tensor(batch.value)

        returns = []
        exp_weighted_return = 0

        for reward, done in zip(rewards[::-1], dones[::-1]):
            exp_weighted_return = reward + discount_factor * exp_weighted_return * (1 - int(done))
            returns += [exp_weighted_return]

        returns = returns[::-1]

        with tf.GradientTape() as tape:
            state_values = self.critic_network(states[None,:])
            critic_loss_value = huber_loss(returns, state_values)

        gradients = tape.gradient(critic_loss_value, self.critic_network.trainable_variables)
        optimizer.apply_gradients(zip(gradients, self.critic_network.trainable_variables))

        with tf.GradientTape() as tape:
            returns = tf.reshape(returns, [batch_size, 1])
            advantages = returns - values

            actions = tf.cast(actions, tf.int32)
            logits = self.actor_network(states[None,:])
            policy_loss_value = wsce_loss(actions, logits, sample_weight=advantages)

            probs = tf.nn.softmax(logits)
            entropy_loss_value = tf.keras.losses.categorical_crossentropy(probs, probs)
            policy_total_loss_value = policy_loss_value - entropy_c * entropy_loss_value

        gradients = tape.gradient(policy_total_loss_value,
                                  self.actor_network.trainable_variables)
        optimizer.apply_gradients(zip(gradients, self.actor_network.trainable_variables))

    def train(self,
              n_steps: int = None,
              n_episodes: int = None,
              save_every: int = None,
              save_path: str = None,
              callback: callable = None,
              **kwargs) -> float:
        batch_size: int = kwargs.get('batch_size', 128)
        discount_factor: float = kwargs.get('discount_factor', 0.9999)
        learning_rate: float = kwargs.get('learning_rate', 0.0001)
        eps_start: float = kwargs.get('eps_start', 0.9)
        eps_end: float = kwargs.get('eps_end', 0.05)
        eps_decay_steps: int = kwargs.get('eps_decay_steps', 200)
        entropy_c: int = kwargs.get('entropy_c', 0.0001)
        memory_capacity: int = kwargs.get('memory_capacity', 1000)

        memory = ReplayMemory(memory_capacity, transition_type=A2CTransition)
        episode = 0
        steps_done = 0
        total_reward = 0
        stop_training = False

        if n_steps and not n_episodes:
            n_episodes = np.iinfo(np.int32).max

        print('====      AGENT ID: {}      ===='.format(self.id))

        while episode < n_episodes and not stop_training:
            state = self.env.reset()
            done = False
            steps_done = 0
            if episode:
                self.critic_network.reset_states()
                self.actor_network.reset_states()
                memory = ReplayMemory(memory_capacity, transition_type=A2CTransition)
            print(self.env.portfolio.balances)
            print('====      EPISODE ID ({}/{}): {}      ===='.format(episode + 1,
                                                                      n_episodes,
                                                                      self.env.episode_id))

            while not done:
                if steps_done % 24 == 0: #each day
                    print("step {}/{}".format(steps_done, n_steps))
                    print(self.env.portfolio.balances)
                    print(self.env.portfolio.net_worth)

                if not self.env.feed.has_next():
                    done = True
                    continue

                threshold = eps_end + (eps_start - eps_end) * np.exp(-steps_done / eps_decay_steps)
                action = self.get_action(state, threshold=threshold)
                next_state, reward, done, _ = self.env.step(action)
                value = self.critic_network(state[None, None, :], training=False)
                value = tf.squeeze(value, axis=-1)

                memory.push(state, action, reward, done, value)

                state = next_state
                total_reward += reward
                steps_done += 1

                if self.env.portfolio.net_worth < self.env.portfolio.initial_net_worth * 0.3:
                    done = True
                    continue

                if len(memory) < batch_size:
                    continue

                if True or steps_done % batch_size == 0:
                    self._apply_gradient_descent(memory,
                                             batch_size,
                                             learning_rate,
                                             discount_factor,
                                             entropy_c)

                if n_steps and steps_done >= n_steps:
                    done = True
                    #stop_training = True

            is_checkpoint = save_every and episode % save_every == 0

            if save_path and (is_checkpoint or episode == n_episodes):
                self.save(save_path, episode=episode)

            episode += 1

        mean_reward = total_reward / steps_done

        return mean_reward



agent = A2C_LSTMAgent_DEV(env)

agent.train(n_steps=data.shape[0], batch_size=24*7, n_episodes=500, save_path="D:/Users/suuser/Desktop")

====      AGENT ID: 55b7f2ed-8101-4a2f-9378-f2f5ae750d57      ====
[10000.00 USD, 0.00000000 BTC, 0.00000000 ETH]
====      EPISODE ID (1/500): d61f2802-b044-4346-b517-b6edbf1288f2      ====
step 0/25272
[10000.00 USD, 0.00000000 BTC, 0.00000000 ETH]
10000.0
step 24/25272
[4670.81 USD, 1.54383352 BTC, 5.22257171 ETH]
9852.1293574378
step 48/25272
[8289.05 USD, 0.63804000 BTC, 0.00000000 ETH]
9885.617492
step 72/25272
[3278.94 USD, 2.53246629 BTC, 0.22727471 ETH]
9965.342181066599
step 96/25272
[2622.53 USD, 2.65282336 BTC, 1.10860108 ETH]
9738.0556980852
step 120/25272
[6593.66 USD, 1.03568019 BTC, 1.50115342 ETH]
9698.6541855872
step 144/25272
[9282.27 USD, 0.07622466 BTC, 0.23427595 ETH]
9536.608665019501
step 168/25272
[1524.55 USD, 0.00000000 BTC, 32.23785461 ETH]
9224.5615735985
step 192/25272
[7113.53 USD, 0.84367365 BTC, 0.00000000 ETH]
9267.715677491
step 216/25272
[0.00 USD, 0.21834570 BTC, 35.38110873 ETH]
8660.818851590699
step 240/25272
[3207.52 USD, 2.00299819 BTC, 0.02419

In [ ]:
print(data.shape)
